# 파이썬으로 메일 보내기

## 메일 서버 접속해서 로그인 후 메일 보내기

### 기본 개념

메일을 보내는 코드를 작성하기 위해서 알아야 할 최소한의 개념부터 살펴보자. 우선 우리가 메일을 보낼 때 서버에서 실제로 일어나는 일을 이해해야 한다.

#### SMTP
https://ko.wikipedia.org/wiki/%EA%B0%84%EC%9D%B4_%EC%9A%B0%ED%8E%B8_%EC%A0%84%EC%86%A1_%ED%94%84%EB%A1%9C%ED%86%A0%EC%BD%9C

우리가 메일을 보내는 행위는 결국 메일 서비스를 제공하는 회사에게 "이런 주소로 이런 내용을 담아 발송해줘"라는 요청을 하는 것이고, 이 회사는 자신의 서버를 통해 발신자의 요청사항을 처리해주게 되는데, 이 때 요청을 처리하는 서버를 SMTP 서버라고 부른다. (SMTP는 Simple Mail Transfer Protocol의 약자로, 이메일을 주고 받는 표준적인 약속 체계나 규약을 의미한다. 즉, 우리가 SMTP 서버에게 미리 약속된 형식으로 요청을 해야 서버에서 그 요청을 처리한다는 뜻이다.)

서버에게 메일 발송 요청을 하려면 서버의 주소와 메일 발송에 사용할 포트(경로)만 알면 된다.

대표적인 메일 서비스들의 주소와 포트를 나열하면 다음과 같다.
- 구글: smtp.gmail.com (465)
  - https://myaccount.google.com/u/4/lesssecureapps (보안 수준이 낮은 앱 허용: 사용)
- 네이버: smtp.naver.com (465)
  - https://help.naver.com/support/contents/contents.help?serviceNo=2342&categoryNo=2281
- 다음: smtp.daum.net (465)
  - https://cs.daum.net/faq/266/12146.html?page=1#18007

465이 아닌 587 포트를 사용하기도 하는데, 본 강의에서는 465로 실습을 진행한다. (465는 SSL, 587은 TLS라는 암호화 프로토콜을 사용하는데, 이에 대한 설명은 생략한다.)

만약 회사 메일서버를 사용하고 싶다면 마찬가지로 SMTP 서버 주소와 포트를 알아내어 적용하면 된다.

(참고로 SMTP 설정 과정에서 IMAP/POP3라는 단어도 많이 마주하게 되는데, SMTP가 메일 발송을 위한 프로토콜이었다면, IMAP/POP3는 메일 수신을 위한 프로토콜이라 이해하면 된다.)

<img src="https://media.vlpt.us/images/snowman39/post/4d3dbf46-0e2b-4242-8f8d-aa72c17f3f74/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202020-08-26%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%205.05.53.png">

#### MIME
https://ko.wikipedia.org/wiki/MIME

MIME(Multipurpose Internet Mail Extensions)은 메일 작성을 위한 표준적인 형식을 의미한다. 우리가 메일 서버에게 "이런 내용으로 메일을 작성해서 보내줘"라고 요청할 때는 서버가 이해할 수 있는 MIME 형식으로 메일을 작성해서 요청해야 한다는 뜻이다.

결국 요약하자면 메일 발송을 위해 준비할 것은 SMTP 서버 주소와 포트, 이메일 계정과 비밀번호 뿐이다. 이후에 파이썬 코드를 통해 메일과 관련한 정보들을 MIME이라는 템플릿에 담아 서버에게 발송을 요청하게 된다. (파이썬 기본으로 내장된 라이브러리만으로 구현이 가능하기 때문에 별도의 라이브러리는 설치하지 않는다.)

### 코드 예제

일단 아래 코드를 처음부터 이해하기에는 머리가 아플 수 있다. 일단 코드를 실행해서 메일이 정상적으로 발송되는지 확인하자.

In [1]:
from myaccount import MY_ID, MY_PW
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from smtplib import SMTP_SSL

# 메일 서버 주소 및 포트
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

# 메일 계정 및 비밀번호
SMTP_USER = MY_ID
SMTP_PASSWORD = MY_PW

# MIME 템플릿 생성
message = MIMEMultipart("alternative")

# 보내는 사람 / 받는 사람 / 제목 기입
message["From"] = SMTP_USER
message["To"] = "hleecaster@gmail.com"
message["Subject"] = "제목"

# "본문파트" 구성
contents = "본문" 
본문파트 = MIMEText(_text=contents, _subtype='plain', _charset="utf-8")
# 메일에 "본문파트" 첨부
message.attach(본문파트)

# SMTP 서버 로그인
smtp = SMTP_SSL(SMTP_SERVER, SMTP_PORT)
smtp.login(SMTP_USER, SMTP_PASSWORD)
# 메일 발송
smtp.sendmail(SMTP_USER, "hleecaster@gmail.com", message.as_string())
# 종료
smtp.close()

그렇다면 이번에는 이메일을 보내는 함수를 만들어서 사용해보자.

In [2]:
from account import MY_ID, MY_PW
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from smtplib import SMTP_SSL

# 메일 서버 주소 및 포트
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

# 메일 계정 및 비밀번호
SMTP_USER = MY_ID
SMTP_PASSWORD = MY_PW

def send_mail(받는사람, 제목, 본문):

    # MIME 템플릿 생성
    message = MIMEMultipart("alternative")
    
    # 보내는 사람 / 받는 사람 / 제목 기입
    message["From"] = SMTP_USER
    message["To"] = 받는사람
    message["Subject"] = 제목

    # "본문파트" 구성
    contents = 본문 
    본문파트 = MIMEText(_text=contents, _subtype='plain', _charset="utf-8")
    # 메일에 "본문파트" 첨부
    message.attach(본문파트)
    
    # SMTP 서버 로그인
    smtp = SMTP_SSL(SMTP_SERVER, SMTP_PORT)
    smtp.login(SMTP_USER, SMTP_PASSWORD)
    # 메일 발송 
    smtp.sendmail(SMTP_USER, 받는사람, message.as_string())
    # 종료
    smtp.close()

In [3]:
send_mail("hleecaster@gmail.com", "제목입니다", "본문입니다")

## 메일 본문 HTML 형식으로 보내기

앞선 예제는 이메일 본문에 서식이 없는 plain 텍스트 형식이다.

그러나 만약 웹페이지처럼 화려한 이메일 본문을 작성하고 싶다면, 그리고 HTML이라는 웹페이지의 골격을 구성하는 언어를 이해하고 있다면 이메일 본문을 HTML 작성해서 보내는 것도 가능하다.

### 코드 예제

앞선 코드에서 MIMEText()의 _subtype을 "plain이 아닌 "html"로 지정한다.

In [4]:
from myaccount import MY_ID, MY_PW
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from smtplib import SMTP_SSL

# 메일 서버 주소 및 포트
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

# 메일 계정 및 비밀번호
SMTP_USER = MY_ID
SMTP_PASSWORD = MY_PW

def send_mail(받는사람, 제목, 본문):

    # MIME 템플릿 생성 
    message = MIMEMultipart("alternative")
    
    # 보내는 사람 / 받는 사람 / 제목 기입
    message["From"] = SMTP_USER
    message["To"] = 받는사람
    message["Subject"] = 제목

    # "본문파트" 구성 (_subtype="html")
    contents = 본문 
    본문파트 = MIMEText(_text=contents, _subtype="html", _charset="utf-8")
    # 메일에 "본문파트" 첨부
    message.attach(본문파트)
    
    # SMTP 서버 로그인
    smtp = SMTP_SSL(SMTP_SERVER, SMTP_PORT)
    smtp.login(SMTP_USER, SMTP_PASSWORD)
    # 메일 발송 
    smtp.sendmail(SMTP_USER, 받는사람, message.as_string())
    # 종료
    smtp.close()

In [5]:
본문 = """
<html>
    <head></head>
    <body>
        <h2>안녕하세요!</h2>
        <h3>HTML 테스트 메일입니다.</h3>
        <img src="https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png">
        <p><a href="http://hleecaster.com/">블로그 바로가기</a></p>
    </body>
</html>
"""

In [6]:
send_mail("hleecaster@gmail.com", "제목입니다", 본문)

## 메일에 파일 첨부하여 보내기

파일을 첨부하기 위해서는 MIME 템플릿의 종류가 달라져야 하고, "본문 파트" 외에 "첨부파일 파트"가 추가되어야 한다.

기존에 plain 텍스트 형식으로 메일을 보내는 send_mail() 함수에서 일부 코드를 수정/추가해서 작성해보자.

- 함수의 인자로 "첨부파일"을 추가한다.
- MIMEMultipart 유형을 "mixed"로 지정한다.
- "첨부파트"를 구성한다.
  - 파일을 바이너리로 읽고 base64로 인코딩한다. (https://ko.wikipedia.org/wiki/%EB%B2%A0%EC%9D%B4%EC%8A%A464#MIME)
  - 파일의 헤더(속성)에 파일명을 추가한다.
* 메일에 "첨부파트"를 추가한다.

### 코드 예제

In [7]:
from myaccount import MY_ID, MY_PW
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from smtplib import SMTP_SSL

# 메일 서버 주소 및 포트
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

# 메일 계정 및 비밀번호
SMTP_USER = MY_ID
SMTP_PASSWORD = MY_PW

def send_mail(받는사람, 제목, 본문, 첨부파일=False):

    # MIME 템플릿 생성 (첨부파일이 있으면 mixed)
    message = MIMEMultipart("alternative")
    if 첨부파일:
        message = MIMEMultipart("mixed")
    
    # 보내는 사람 / 받는 사람 / 제목 기입
    message["From"] = SMTP_USER
    message["To"] = 받는사람
    message["Subject"] = 제목

    # "본문파트" 구성
    contents = 본문 
    본문파트 = MIMEText(_text=contents, _subtype='plain', _charset="utf-8")
    # 메일에 "본문파트" 첨부
    message.attach(본문파트)
    
    if 첨부파일:
        # "첨부파트"를 구성하기 위한 라이브러리
        from email.mime.base import MIMEBase
        from email import encoders
        # "첨부파트" 구성
        첨부파트 = MIMEBase("application", "octect-stream")
        # 파일을 바이너리 형태로 읽어서
        첨부파트.set_payload(open(첨부파일, "rb").read())
        # base64 형태로 인코딩
        encoders.encode_base64(첨부파트)
        
        # 실제 파일명을 전달하기 위한 라이브러리
        from pathlib import Path
        # 파일의 전체경로 중 실제 파일명만 가져오고  
        파일명 = Path(첨부파일).name
        # 파일 헤더(설명)으로 파일명을 추가한 후
        첨부파트.add_header("Content-Disposition", "attachment", filename=파일명)
        # 메일에 "첨부파트" 첨부  
        message.attach(첨부파트)
    
    # SMTP 서버 로그인
    smtp = SMTP_SSL(SMTP_SERVER, SMTP_PORT)
    smtp.login(SMTP_USER, SMTP_PASSWORD)
    # 메일 발송 
    smtp.sendmail(SMTP_USER, 받는사람, message.as_string())
    # 종료
    smtp.close()

In [8]:
send_mail("hleecaster@gmail.com", "제목입니다", "본문입니다", "files/첨부1.txt")

## (실습) 제목, 본문, 첨부파일 다르게 대량으로 이메일 보내기

In [9]:
from myaccount import MY_ID, MY_PW
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from smtplib import SMTP_SSL
from openpyxl import load_workbook

# 메일 서버 주소 및 포트
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465
# 메일 계정 및 비밀번호
SMTP_USER = MY_ID
SMTP_PASSWORD = MY_PW

def send_mail(받는사람, 제목, 본문, 첨부파일=False):

    # MIME 템플릿 생성 (첨부파일이 있으면 mixed)
    message = MIMEMultipart("alternative")
    if 첨부파일:
        message = MIMEMultipart("mixed")
    
    # 보내는 사람 / 받는 사람 / 제목 기입
    message["From"] = SMTP_USER
    message["To"] = 받는사람
    message["Subject"] = 제목

    # "본문파트" 구성
    contents = 본문 
    본문파트 = MIMEText(_text=contents, _subtype='plain', _charset="utf-8")
    # 메일에 "본문파트" 첨부
    message.attach(본문파트)
    
    if 첨부파일:
        # "첨부파트"를 구성하기 위한 라이브러리
        from email.mime.base import MIMEBase
        from email import encoders
        # "첨부파트" 구성
        첨부파트 = MIMEBase("application", "octect-stream")
        # 파일을 바이너리 형태로 읽어서
        첨부파트.set_payload(open(첨부파일, "rb").read())
        # base64 형태로 인코딩
        encoders.encode_base64(첨부파트)
        
        # 실제 파일명을 전달하기 위한 라이브러리
        from pathlib import Path
        # 파일의 전체경로 중 실제 파일명만 가져오고  
        파일명 = Path(첨부파일).name
        # 파일 헤더(설명)으로 파일명을 추가한 후
        첨부파트.add_header("Content-Disposition", "attachment", filename=파일명)
        # 메일에 "첨부파트" 첨부  
        message.attach(첨부파트)
    
    # SMTP 서버 로그인
    smtp = SMTP_SSL(SMTP_SERVER, SMTP_PORT)
    smtp.login(SMTP_USER, SMTP_PASSWORD)
    # 메일 발송 
    smtp.sendmail(SMTP_USER, 받는사람, message.as_string())
    # 종료
    smtp.close()

    print(받는사람, "OK", sep="\t")
    
    
# 메일 리스트가 담긴 엑셀파일
wb = load_workbook('files/이메일리스트.xlsx', data_only=True)
ws = wb["명단"]

for row in ws.iter_rows():
    받는사람 = row[0].value
    제목 = row[1].value
    본문 = row[2].value
    첨부파일 = row[3].value
    send_mail(받는사람, 제목, 본문, 첨부파일)